In [12]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, SimpleRNN, LSTM, GRU, Dense

In [13]:
num_words= 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

In [14]:
maxlen = 100
x_train = pad_sequences(x_train, maxlen=maxlen, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=maxlen, padding='post', truncating='post')

In [16]:
version_fn = getattr(tf.keras, "version", None)
if version_fn and version_fn().startswith("3."):
    import tensorflow.keras as keras
else:
    keras = tf.keras

In [30]:
from tensorflow.keras.layers import Input
embedding =100
encoder_input = Input(shape=(None,))
model = Sequential([
    encoder_input,
    Embedding(input_dim=num_words, output_dim=embedding, input_length=maxlen),
    Bidirectional(LSTM(5)),
    Dense(1, activation='sigmoid')
])
model.build()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    batch_size=num_words,
                    epochs=1)
model.summary()


c:\Users\HP\Desktop\VSCode\Bidirection_lstm\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 856ms/step - accuracy: 0.5329 - loss: 0.6922


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ (None, None, 100)      │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_6 (Bidirectional) │ (None, 10)             │         4,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,012,755 (11.49 MB)

 Trainable params: 1,004,251 (3.83 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,008,504 (7.66 MB)

In [39]:
train_loss, train_acc = model.evaluate(x_test, y_test, verbose=2)

782/782 - 7s - 8ms/step - accuracy: 0.6047 - loss: 0.6889


In [40]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [41]:
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [42]:
log_dir

'logs/fit20240724-192227'

In [43]:
model_callbacks = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [38]:
import mlflow

In [46]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, callbacks=[model_callbacks, earLy_stop]
)

Epoch 1/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 42s 53ms/step - accuracy: 0.7332 - loss: 0.5441 - val_accuracy: 0.8086 - val_loss: 0.4231
Epoch 2/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 41s 53ms/step - accuracy: 0.8809 - loss: 0.3025 - val_accuracy: 0.8105 - val_loss: 0.4195
Epoch 3/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 39s 49ms/step - accuracy: 0.9207 - loss: 0.2155 - val_accuracy: 0.8002 - val_loss: 0.4987
Epoch 4/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 38s 49ms/step - accuracy: 0.9478 - loss: 0.1488 - val_accuracy: 0.7930 - val_loss: 0.5895
Epoch 5/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 41s 53ms/step - accuracy: 0.9642 - loss: 0.1050 - val_accuracy: 0.7904 - val_loss: 0.6108
Epoch 6/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 40s 52ms/step - accuracy: 0.9691 - loss: 0.0921 - val_accuracy: 0.7934 - val_loss: 0.7166
Epoch 7/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 40s 51ms/step - accuracy: 0.9860 - loss: 0.0513 - val_accuracy: 0.7860 - val_loss: 0.7766
Epoch 8/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 49s 62ms/step - accuracy: 0.9852 - loss: 0

In [45]:
earLy_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [47]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ (None, None, 100)      │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_6 (Bidirectional) │ (None, 10)             │         4,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,012,755 (11.49 MB)

 Trainable params: 1,004,251 (3.83 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,008,504 (7.66 MB)

In [48]:
from mlflow.models.signature import infer_signature
model_sign = infer_signature(x_test, model.predict(x_test))

782/782 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step


In [49]:
exp_timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
exp_name= "IMDB_bidirection_lstm_time:" + exp_timestamp
#exp_name = "IMDB_bidirection_lstm"
run_timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
run_name = "IMDB_bidirection_lstm_run:" + run_timestamp

In [51]:
def tf_experiment_tracking(exp_name, run_name, train_loss, train_acc, model_sign):
    mlflow.set_experiment(exp_name)
    with mlflow.start_run(run_name=run_name):
        mlflow.log_param("batch_size", num_words )
        mlflow.log_param("learning_rate", 0.001)
        mlflow.log_param("epochs", 100)
        mlflow.log_metric("train_loss", train_loss)
        mlflow.log_metric("train_accuracy", train_acc)
        mlflow.tensorflow.log_model(model, "IMDB", signature=model_sign)
    mlflow.end_run()

In [52]:
tf_experiment_tracking(exp_name, run_name, train_loss, train_acc, model_sign)

2024/07/24 19:38:42 INFO mlflow.tracking.fluent: Experiment with name 'IMDB_bidirection_lstm_time:20240724-193146' does not exist. Creating a new experiment.


In [59]:
!mkdir -p saved_model

In [53]:
model.export(filepath='saved_model', format='tf')

INFO:tensorflow:Assets written to: saved_model\assets


INFO:tensorflow:Assets written to: saved_model\assets


Saved artifact at 'saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, None), dtype=tf.float32, name='keras_tensor_19')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2500354831568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2500354829072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2500354829456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2500354820432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2500354821392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2500354821776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2500354830032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2500354829264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2500354821008: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [59]:
from pathlib import Path
from service.bentoml_load_model import load_model_and_save_it_in_bentoml


In [70]:
import bentoml
model.summary()
bentoml.tensorflow.save_model('bento_model',  model,
signatures={"__call__": {"batchable": True, "batch_dim": 0}})

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ (None, None, 100)      │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_6 (Bidirectional) │ (None, 10)             │         4,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,012,755 (11.49 MB)

 Trainable params: 1,004,251 (3.83 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,008,504 (7.66 MB)

TypeError: this __dict__ descriptor does not support '_DictWrapper' objects